# Get Chef's Recommendations

Build a Telegram bot using Spoonacular APIs to get recipes based on a list of ingredients. The user sends commands to the epicurean bot. The command `/joke` will give the joke of the day. The command `/recipe` with a list of ingredients with return a recipe-card with the best match found.

## Steps

The steps will involve:
- Set up a Telegram bot with name and username as required.
- Initialize the bot with access token and get vital statistics.
- Instantiate the key classes for implementing our epicure's behavior.
- Implement the `/start` callback to give help.

And then:
- Implement `/joke`.
- Implement `/recipe` with list of ingredients.

### Set up Telegram

Set up a telegram bot using the Botfather. Call it **Epicurean** with username *epicurryin_bot*.

This gave
```
Done! Congratulations on your new bot. You will find it at t.me/epicurryin_bot. You can now add a description, about section and profile picture for your bot, see /help for a list of commands. By the way, when you've finished creating your cool bot, ping our Bot Support if you want a better username for it. Just make sure the bot is fully operational before you do this.

Use this token to access the HTTP API:
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

Keep your token secure and store it safely, it can be used by anyone to control your bot.

For a description of the Bot API, see this page: https://core.telegram.org/bots/api
```

### Initialize the bot with access token and get vital stats

In [1]:
import telegram
import config

In [2]:
epicurean = telegram.Bot(token=config.token)

In [3]:
print(epicurean.get_me())

{'id': 1150777436, 'first_name': 'Epicurean', 'is_bot': True, 'username': 'epicurryin_bot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': False}


### Initialize the key classes for implementing Epicurean

We will need the following:
- `Updater` for dispatch-routing callbacks
- `CommandHandler` for implementing Epicurean's behavior in response to commands 
- `Filters` for sifting through the message queue (containing messages and commands)
- `MessageHandler` for implementing Epicurean's response to text messages
- Other

These are from `telegram.ext`. Remember that dispatcher receives messages (including text messages and commands) from updater via queue. The dispatcher then sorts through the messages and passes them to the handler with the matching signature. We will implement a fall-back for unknown command.

In [4]:
from telegram.ext import Updater, CommandHandler, Filters, MessageHandler

In [5]:
updater = Updater(token=config.token, use_context=True)
dispatcher = updater.dispatcher

### Give a helpful message at `/start`

Implement the `start()` function, set as the `/start` command's callback and register the handler with dispatcher. Give the user a condensed user manual listing commands with usage samples. 

In [6]:
def start(update, context):
    update.message.reply_text(
        'I currently accept the following:\n'
        'Send /joke for epicurean joke-of-the-day. '
        'Send /recipe with list of ingredients for recipe card.\n\n'
        'For example:\n'
        '/joke\n'
        '/recipe cucumber curd\n\n'
        'Capisce?'
    )

In [7]:
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

### Start the bot

From this point on until `stop()`, the bot is running and will engage users.

In [8]:
updater.start_polling()

## Implement joke-of-the-day with `/joke`

We will use the Spoonacular API. We have set up a free account with gmail and password. The API call for joke of the day is [here](https://api.spoonacular.com/food/jokes/random). Use the `requests` library to make a GET call.

### Implement wrapper around API end-point

Wrap a function around the API end-point and test.

In [9]:
import requests
import json

In [10]:
def joke_du_jour():
    """
    Get a random joke from the Spoonacular API.
    
    Input args: None
    
    Returned value: (String) A line of text
    
    Sample usage:
    >>> joke_du_jour()
    """
    url_joke = 'https://api.spoonacular.com/food/jokes/random'

    joke = None
    
    packet = {
        'apiKey': config.apiKey
    }
    
    r = requests.get(url_joke, params=packet)
    print("Status: {} & Reason: {}".format(r.status_code, r.reason))
    
    if r.status_code == 200:
        joke = json.loads(r.text)["text"]
        
    return joke

In [11]:
just_kidding = joke_du_jour()
print(just_kidding)

Status: 200 & Reason: OK
I love the way you move...like butter on a bald monkey.


### Tell what's funny

Add call-back to the handler and register with dispatcher. You know the drill.

In [12]:
def joke(update, context):
    punch_line = joke_du_jour()
    context.bot.send_message(chat_id=update.effective_chat.id, text=punch_line)

In [13]:
joke_handler = CommandHandler('joke', joke)
dispatcher.add_handler(joke_handler)

## Whip up a recipe given ingredients

We will do this in the following steps, with knowledge of the [Spoonacular API](https://spoonacular.com/food-api/docs). Use th following end-points to build the recipe step-step:
1. Search recipe by ingredients [here](https://spoonacular.com/food-api/docs#Search-Recipes-by-Ingredients)
2. Get full recipe information with ID [here](https://spoonacular.com/food-api/docs#Get-Recipe-Information)
3. Generate recipe card [here](https://spoonacular.com/food-api/docs#Create-Recipe-Card)

Here, each API end-point furnishes information for the next step. This approach is driven by the architecture or the Spoonacular API.

### Search for a recipe by available ingredients

We will make a GET request to the API end-point, parse out information and return a tuple with several items.

In [14]:
def searchRecipeByIngredients(ingredients_list):
    """
    Fetch the ID and key information from the first hit.
    Input args:
    - ingredients_list: (list of strings) a list of ingredients to search by 
    
    Returned value:
    - ID: (integer) the numeric ID of the hit, if found (None otherwise)
    - Title: (string) the title of the recipe, if found (None otherwise)
    - Img: (string) the URL of an image of the recipe, if found (None otherwise)
    - Raw: (string) the plain-JSON response of API end-point, if 200 OK (None otherwise)
    
    Sample usage:
    >>> Id, Title, Img, Raw = searchRecipeByIngredients(['spinach', 'tomato', 'chillies', 'cottage cheese'])
    """
    url_search = "https://api.spoonacular.com/recipes/findByIngredients"
    
    recipe_raw = None
    recipe_ID = None
    recipe_title = None
    recipe_image = None
    
    packet = {
        'apiKey': config.apiKey,
        'ingredients': ingredients_list,
        'number': 1
    }
    r = requests.get(url_search, params=packet)
    print("Status: {}, Reason: {}".format(r.status_code, r.reason))
    
    if r.status_code == 200:
        recipe_raw = r.text
        recipe_json = json.loads(r.text)
        recipe_ID = recipe_json[0]["id"]
        recipe_title = recipe_json[0]["title"]
        recipe_image = recipe_json[0]["image"]
        
        
    return (recipe_ID, recipe_title, recipe_image, recipe_raw)

In [15]:
Id, Title, Img, Raw = searchRecipeByIngredients(['paneer', 'palak'])

Status: 200, Reason: OK


In [16]:
print("Got {} named {}.".format(Id, Title))
print("Download here: {}".format(Img))

Got 652592 named Multigrain Tandoori Pizza With Paneer Tikka.
Download here: https://spoonacular.com/recipeImages/652592-312x231.jpg


*So far we have located a recipe that uses the ingredients we have furnished. The result only has information like ID and title. Next, we will use the ID to get more details.*

### Get detailed information about a recipe

We will use the recipe ID to get more information, parse out items to stash in a tuple and return it. 

In [17]:
def getRecipeInformation(recipe_id):
    """
    Input args:
    - recipe_id: (integer) the numeric ID of the recipe in the Spoonacular database.
    
    Returned value(s):
    - title: (string) the name of the recipe
    - image: (string) a photo of the dish
    - ingredients: (list of strings) a list of ingredients needed
    - instructions: (string) the recipe instructions
    - ready_in_minutes: (integer) time to cook in minutes
    - servings: (integer) the number of servings 
    - info: (string) raw information 
    
    Sample usage:
    >>> title, image_url, ingredients, instructions, cook_time, servings, info = getRecipeInformation(652592)
    """
    url_info = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    
    title = None
    image = None
    ingredients = []
    instructions = None
    ready_in_minutes = 0
    servings = 0
    info = None
    
    packet = {
        'apiKey': config.apiKey,
    }
    r = requests.get(url_info, params=packet)
    print("Status: {}, Reason: {}".format(r.status_code, r.reason))
    
    if r.status_code == 200:
        info_json = json.loads(r.text)
        title = info_json["title"]
        image = info_json["image"]
        ingredients = [x["name"] for x in info_json["extendedIngredients"]]
        instructions = info_json["instructions"]
        ready_in_minutes = info_json["readyInMinutes"]
        servings = info_json["servings"]
        info = r.text
        
    return (title, image, ingredients, instructions, ready_in_minutes, servings, info)

In [18]:
title, image_url, ingredients, instructions, cook_time, servings, info = getRecipeInformation(652592)

Status: 200, Reason: OK


Unpack the information and print it. The ID, title, image, list of ingredients, set of instructions and cooking time for a number of services are all required in order to generate a recipe card. We will use the parcels of information that the function returns in this test and examine it.

In [19]:
print("id: {}".format(652592))
print("title: {}".format(title))
print("image URL: {}".format(image_url))
print("ingredients: {}".format(ingredients))
print("instructions: {}".format(instructions))
print("ready_in_minutes: {}".format(cook_time))
print("servings: {}".format(servings))

id: 652592
title: Multigrain Tandoori Pizza With Paneer Tikka
image URL: https://spoonacular.com/recipeImages/652592-556x370.jpg
ingredients: ['garam masala', 'paneer', 'parsley', 'pizza dough', 'salt']
instructions: <ol><li>Heat olive oil in a pan and add ginger garlic, salt, paprika with a dash of garam masala, saute for few seconds.</li><li>In a bowl, whip the yogurt and add the seasoning prepared above to the yogurt.</li><li>If you want to make the Tadoori Sauce Spicy, you can add some freshly chopped green chilly & can also add some fresh cilantro/parsley to add some flavor. Mix well & our Tandoor sauce is ready.</li><li>Cut Paneer and all vegetables into cube chunks approx 1inch X 1 inch.</li><li>Divide Tandoori sauce in to two bowls.</li><li>Now add Paneer and vegetable into one bowl.</li><li>Mix well and add bell pepper, tomato, onion and Paneer chunks.</li><li>Cover the bowl and keep in refrigerator for at least 45-60 minutes.</li><li>Take the 2nd bowl of tandoori sauce & gene

We have all the information that is required for a recipe card. However, further processing is required to structue this information to make a valid request. Get recipe information with ID 541025.

In [20]:
title_541025, image_url_541025, ingredients_541025, instructions_541025, cook_time_541025, servings_541025, info_541025 = getRecipeInformation(541025)

Status: 200, Reason: OK


In [21]:
print("id: {}".format(541025))
print("title: {}".format(title_541025))
print("image URL: {}".format(image_url_541025))
print("ingredients: {}".format(ingredients_541025))
print("instructions: {}".format(instructions_541025))
print("ready_in_minutes: {}".format(cook_time_541025))
print("servings: {}".format(servings_541025))

id: 541025
title: Karaage Bento
image URL: https://spoonacular.com/recipeImages/541025-556x370.jpg
ingredients: ['broccoli', 'spinach', 'tomatoes', 'karaage', 'onigiri', 'tamagoyaki']
instructions: Take leftover Karaage out of fridge and bake for a few minutes in a toaster oven until it's warm.Meanwhile make Onigiri and put it in the bento box.Saute spinach and corn and pack in the bento box..Place Karaage in the bento box.Put Tamagoyaki in the box.Wash tomatoes and pat dry. Place nicely in the bento box along with thawed broccoli.Cool down completely before closing the bento box.
ready_in_minutes: 20
servings: 1


The API docs tell us to provide the ingredients and instructions as words separated using the symbol `\n` (i.e. a back-slash followed by the lower-case letter 'n') as spacer. Format the data accordingingly. Use `join()` to create a string in the requested format from the list of ingredients like so.

In [22]:
print("ID {}: {}".format(652592, "\\n".join(ingredients)))
print("ID {}: {}".format(541025, "\\n".join(ingredients_541025)))

ID 652592: garam masala\npaneer\nparsley\npizza dough\nsalt
ID 541025: broccoli\nspinach\ntomatoes\nkaraage\nonigiri\ntamagoyaki


Note that there is free-text instructions and a slot for analyzed instructions. The free-text instructions may present special characters and other problems. Therefore, we will use analyzed instructions which are clean. Navigate the json with the dictionary keys from `keys()` command and the `get()` command get the analyzed instructions safely. 

In [23]:
info
info_json = json.loads(info)
info_keys = info_json.keys()
analyzed_instructions = info_json.get("analyzedInstructions", None)
analyzed_steps = [x["step"] for x in analyzed_instructions[0]["steps"]]
print("ID {}:\n {}".format(652592, "\n".join(analyzed_steps)))

ID 652592:
 Heat olive oil in a pan and add ginger garlic, salt, paprika with a dash of garam masala, saute for few seconds.In a bowl, whip the yogurt and add the seasoning prepared above to the yogurt.If you want to make the Tadoori Sauce Spicy, you can add some freshly chopped green chilly & can also add some fresh cilantro/parsley to add some flavor.
Mix well & our Tandoor sauce is ready.
Cut Paneer and all vegetables into cube chunks approx 1inch X 1 inch.Divide Tandoori sauce in to two bowls.Now add Paneer and vegetable into one bowl.
Mix well and add bell pepper, tomato, onion and Paneer chunks.Cover the bowl and keep in refrigerator for at least 45-60 minutes.Take the 2nd bowl of tandoori sauce & generously spread it over the pizza crust.Leave aside for 15 mins as we grill our Paneer Tikka in the meantime.Take a wooden skewer and put veggies in a sequence - bell peppers, onion, tomato, paneer.Repeat the process to make more skewers.
Heat olive a non stick pan,  slowly lay all th

Write a function to wrap around the processing steps. There is a fair amount of work involved. By writing a function we will ensure consistent results across a variety of use-cases.

In [24]:
def presentRecipeInstructions(raw_info):
    """
    Presents the instructions for a recipe in the format required
    by the API end-point for recipe card with recipe details. The 
    individual instructions are separated by the '\n' symbol, i.e.
    a back-slash followed by the letter 'n'.
    Input args:
    - raw_info: (string) raw text response from REST API end-point
    
    Return value: (string) a string with instructions in the specified format.
    
    Sample usage:
    >>> title, image_url, ingredients, instructions, cook_time, servings, infor = getRecipeInformation(652592)
    >>> presentRecipeInstructions(infor)
    
    """  

    instruction_set = "Has no instructions."
    analyzed_instructions = json.loads(raw_info).get("analyzedInstructions", None)
    if len(analyzed_instructions) != 0:
        analyzed_steps = [x["step"] for x in analyzed_instructions[0]["steps"]]
        instruction_set =  "\n".join(analyzed_steps) 
        
    return instruction_set

In [25]:
presentRecipeInstructions(info_541025)

"Take leftover Karaage out of fridge and bake for a few minutes in a toaster oven until it's warm.Meanwhile make Onigiri and put it in the bento box.\nSaute spinach and corn and pack in the bento box..\nPlace Karaage in the bento box.Put Tamagoyaki in the box.Wash tomatoes and pat dry.\nPlace nicely in the bento box along with thawed broccoli.Cool down completely before closing the bento box."

In [26]:
url_card = "https://api.spoonacular.com/recipes/visualizeRecipe?apiKey=" + config.apiKey

body = {
    "title": "Simple",
    "ingredients": "\n".join(ingredients_541025),
    "instructions": presentRecipeInstructions(info_541025),
    "readyInMinutes": 45,
    "servings": 2,
    "mask": "diamondMask",
    "backgroundImage": "none"   
}

headers = {
    "Content-Type": "multipart/form-data"
}

In [27]:
r = requests.post(url=url_card, data=body, files={'image':open("Sample_Recipe.jpg", 'rb')})
print("Status: {}; Reason: {}".format(r.status_code, r.reason))

Status: 200; Reason: OK


In [28]:
print(json.loads(r.text).get("url", "Hungry hippo!"))

http://webknox.com/recipeCardImages/recipeCard-1591946442985.png


Follow this link to an early [sample recipe card](http://webknox.com/recipeCardImages/recipeCard-1591900730794.png) generated when testing this API end-point. Take a look at [this sample card](http://webknox.com/recipeCardImages/recipeCard-1591924638812.png) with refinements. Or [this one](http://webknox.com/recipeCardImages/recipeCard-1591935798520.png) where the recipe has no instructions.

Our best result so far is [palak panner meets pizza](http://webknox.com/recipeCardImages/recipeCard-1591936354848.png)!

We now have all the pieces to assemble. We will write a wrapper function to take a list of ingredients and return a beautiful recipe card. Let's go!

We need a couple of helper functions to procure an image of the recipe and use it. We will generate a random label for the image. We will download the image using `requests` and name it with the random label. 

In [29]:
import random
import string

def nameMe(width = 8):
    letters_and_numbers = string.ascii_letters + string.digits
    return ''.join(random.choice(letters_and_numbers) for i in range(width))

In [30]:
nameMe() + ".jpg"

't3UpO6pG.jpg'

In [31]:
def saveMe(image_url):
    r = requests.get(image_url)
    
    nomme_de_plume = nameMe() + ".jpg"
    with open(nomme_de_plume, 'wb') as f:
        f.write(r.content)
        
    return nomme_de_plume

In [32]:
saveMe("https://spoonacular.com/recipeImages/652592-556x370.jpg")

'l6dIt8tD.jpg'

In [33]:
def getRecipeCard(ingredients_list):
    """
    Gets a recipe card made on the basis of ingredients furnished.
    
    Input args:
    - ingredients_list: (list of strings) a list of ingredients with which to find a recipe.
    
    Return value(s):
    - card_url: (string) a url for the recipe card
    - num_id: (int) the numeric ID of the recipe in Spoonacular's database
    - title: (string) the name of the recipe
    - raw_text: (string) the detailed recipe in raw text
    
    Sample usage:
    >>> card_url, num_id, title, raw_text = getRecipeCard(['spinach', 'tomato', 'chillies', 'cottage cheese'])
    """
    card_url = None
    num_id = None
    title = None
    raw_text = None
    
    num_id, title, img, raw_text = searchRecipeByIngredients(ingredients_list)
    title_, img_url, ingredients, instructions, cook_time, servings, info = getRecipeInformation(num_id)
    
    image_name = saveMe(img_url)
    
    url_card = "https://api.spoonacular.com/recipes/visualizeRecipe?apiKey=" + config.apiKey
    body = {
        "title": title,
        "ingredients": "\n".join(ingredients),
        "instructions": presentRecipeInstructions(info),
        "readyInMinutes": cook_time,
        "servings": servings,
        "mask": "diamondMask",
        "backgroundImage": "none"   
    }
    headers = {
        "Content-Type": "multipart/form-data"
    }
    
    r = requests.post(url=url_card, data=body, files={'image':open(image_name, 'rb')})
    print("Status: {}; Reason: {}".format(r.status_code, r.reason))
    
    card_url = json.loads(r.text).get("url", "https://feed.hamster.com")

    return card_url, num_id, title, info

In [34]:
card_url, num_id, title, raw_text = getRecipeCard(['spinach', 'tomato', 'chillies', 'cottage cheese'])

Status: 200, Reason: OK
Status: 200, Reason: OK
Status: 200; Reason: OK


In [35]:
card_url

'http://webknox.com/recipeCardImages/recipeCard-1591946445771.png'

In [36]:
pp_card_url, pp_num_id, pp_title, pp_raw_text = getRecipeCard(['paneer', 'palak'])

Status: 200, Reason: OK
Status: 200, Reason: OK
Status: 200; Reason: OK


In [37]:
pp_card_url

'http://webknox.com/recipeCardImages/recipeCard-1591946448085.png'

### Conclusion

*So far, we have used the Spoonacular API to creat a recipe card with a tasteful picture of the dish accompanied by title, ingredients, instructions. The result is a URL to the recipe card. Next, we will attach the code to our Happy-Curry'in bot.*

## Happy Curry'in with Epicurean

Write a `recipe()` function that implements the call-back routine. Use `context.args` as show in [Your first Bot](https://github.com/python-telegram-bot/python-telegram-bot/wiki/Extensions-–-Your-first-Bot). Refer to the [code snippet](https://github.com/python-telegram-bot/python-telegram-bot/wiki/Code-snippets#post-a-photo-from-a-url) for posting a photo from URL. Lastly, we will handle any unknow commands gracefully.

In [38]:
def recipe(update, context):
    image_url = "http://webknox.com/recipeCardImages/recipeCard-1591936354848.png"
    pp_card_url, pp_num_id, pp_title, pp_raw_text = getRecipeCard(context.args)
    if pp_card_url:
        image_url = pp_card_url
    context.bot.send_photo(chat_id=update.effective_chat.id, photo=image_url)

In [39]:
recipe_handler = CommandHandler('recipe', recipe)
dispatcher.add_handler(recipe_handler)

In [40]:
def unknown(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="No soup for you!")

In [41]:
unknown_handler = MessageHandler(Filters.command, unknown)
dispatcher.add_handler(unknown_handler)

In [42]:
updater.stop()

## TO-DO LIST

1. Add logging
2. Handle unknown commands
3. Make it conversational